In [1]:
import array, numpy as np

# array.array('B') are unsigned char in C

def vernam_operation(src_array,keystream):
    key_len = len(keystream)
    return array.array('B',[np.bitwise_xor(v,keystream[i % key_len]) for i,v in enumerate(src_array)])

def bytearray_to_readable(btarr):
    return " ".join([hex(b)[2:].upper().zfill(2) for b in btarr])

class Vernam:
    def __init__(self,keystream):
        self.keystream = keystream
    @staticmethod
    def keygen(length):
        gen = np.random.default_rng()
        keystream = array.array('B')
        bitlen = 2 ** (keystream.itemsize * 8)
        for _ in range(length):
            keystream.append(gen.integers(0,bitlen))
        return keystream
    @staticmethod
    def bytearray_to_string(arr):
        return "".join([chr(v) for v in arr])
    @staticmethod
    def string_to_bytearray(chars):
        return array.array('B',[ord(v) for v in chars])
    def operation(self,a):
        return vernam_operation(a,self.keystream)
    # Reverse of XOR is also XOR
    encrypt = operation
    decrypt = operation

In [2]:
key = Vernam.keygen(10)
print(key)

array('B', [30, 154, 248, 24, 214, 74, 3, 56, 217, 85])


In [3]:
ve = Vernam(key)
m = "We are discovered, safe yourself."
m_btarr = Vernam.string_to_bytearray(m)
c = ve.encrypt(m_btarr)
rtn_m_btarr = ve.decrypt(c)
rtn_m = Vernam.bytearray_to_string(rtn_m_btarr)

print(m,bytearray_to_readable(m_btarr))
print(bytearray_to_readable(c))
print(rtn_m,bytearray_to_readable(rtn_m_btarr))

We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E
49 FF D8 79 A4 2F 23 5C B0 26 7D F5 8E 7D A4 2F 67 14 F9 26 7F FC 9D 38 AF 25 76 4A AA 30 72 FC D6
We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E


## One-time Pad (OTP)

In [4]:
def OTP_encrypt(m):
    key = Vernam.keygen(len(m))
    return Vernam(key).encrypt(m), key

def OTP_decrypt(c,key):
    return Vernam(key).decrypt(c)

for i in range(5):
    print("Run {}".format(i + 1))
    m = "We are discovered, safe yourself."
    m_btarr = Vernam.string_to_bytearray(m)
    c, key = OTP_encrypt(m_btarr)
    rtn_m_btarr = OTP_decrypt(c, key)
    rtn_m = Vernam.bytearray_to_string(rtn_m_btarr)

    print(bytearray_to_readable(key))
    print(m,bytearray_to_readable(m_btarr))
    print(bytearray_to_readable(c))
    print(rtn_m,bytearray_to_readable(rtn_m_btarr))

Run 1
3C 6B ED 27 8B A5 7D E8 51 C4 19 3B 6A 2B A9 06 39 25 F8 FF A4 93 71 68 C8 E5 ED 30 F8 33 B8 CE AB
We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E
6B 0E CD 46 F9 C0 5D 8C 38 B7 7A 54 1C 4E DB 63 5D 09 D8 8C C5 F5 14 48 B1 8A 98 42 8B 56 D4 A8 85
We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E
Run 2
73 6C C4 75 8F 9F 1E FB A3 92 AC E1 1D D1 68 62 47 D6 90 0D 02 12 31 E4 C3 4F 3F 12 E2 06 5A FB B5
We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E
24 09 E4 14 FD FA 3E 9F CA E1 CF 8E 6B B4 1A 07 23 FA B0 7E 63 74 54 C4 BA 20 4A 60 91 63 36 9D 9B
We are discovered, safe yourself. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E
Run 3
FC 1F 1E 16 E6 60 79 24 DC 2F 20 84 C8 F3 26 D7 1B E4 